# Calling a C program

This will show how to call a compiled C program from Python.  The example program finds the maximum entry in an array.

It will also do a speed comparison of the following:

* Custom code written in Python to do the same thing
* Calling the C code
* using a built in numpy method to do it.

Building the dynamic library

* On a Mac
    * gcc -dynamiclib -o libfind_max.dylib find_max.c
* On Linux
    * gcc -fPIC -shared -o libfind_max.so find_max.c
* On Windows
    * Install free version of Visual Studio 2022, including C/C++ development tools
    * Open a command window associated with the 64-bit compiler
    * cl /LD find_max.c


In [2]:
# Create a numpy array with 1000 random numbers between 0 and 1
import numpy as np

# Create 1000 random floats between 0 and 1  
arr = np.random.rand(1000).astype(np.double) # This forces the type to be double (float64). MUST match C routine!

# Import the C library file

Instructions on creating the library are found in the header of "find_max.c"

In [3]:
# Import a compiled C routine to find the max
import ctypes
import os
import platform

# Load the shared library.  Name will be different on different OSs
system = platform.system()
if system=="Darwin":           # MacOS
    file = "libfind_max.dylib"
elif system=="Windows":        # Windows
    # For some reason, need the full path on Windows (or maybe add to path?)
    file = "find_max.dll"
else:                          # Must be Linux
    file = "libfind_max.so"

# Load the library. Must use the absolute path to the library file
fullpath = os.path.abspath(file)
print(f"Loading library {fullpath}")
lib = ctypes.CDLL(fullpath)

# Tell ctypes about the function signature
# Argument types
lib.find_max.argtypes = [ctypes.POINTER(ctypes.c_double), ctypes.c_int]
# Return types
lib.find_max.restype = ctypes.c_double

# C-style pointer to the data in the array.  This data type must match both the 
# numpy array AND the C routine.  No error checking is done at this point.
arr_ptr = arr.ctypes.data_as(ctypes.POINTER(ctypes.c_double))

Loading library /Users/prebys/Box Sync/Teaching/40/phy40/Lecture 5/libfind_max.dylib


Test the program

In [4]:
# Call my C routine
max = lib.find_max(arr_ptr,len(arr))

# Compare it to the build in function
print(f"C function returns:\t{max}")
print(f"Built-in returns:\t{arr.max()}")

C function returns:	0.9998063107516182
Built-in returns:	0.9998063107516182


Write a Python routine to do the same thing

In [5]:
# find max
def find_max(a):
    max = a[0]
    for val in a:
        if val>max:
            max=val
    return max

# Test it
print(f"Custom function returns:\t{find_max(arr)}")

Custom function returns:	0.9998063107516182


Test the speed of all three

In [6]:
import time
ntest = 100000   # Do 100,000 calls

# Python routine
start = time.perf_counter()
for i in range(ntest):
    max = find_max(arr)
end = time.perf_counter()
time_python = end-start

# C routine
start = time.perf_counter()
for i in range(ntest):
    max = lib.find_max(arr_ptr, len(arr))
end = time.perf_counter()
time_C = end-start

# Built in
for i in range(ntest):
    max = arr.max()
end = time.perf_counter()
time_numpy=end-start

print(f"Execution times for {ntest} calls.\n",
      f"\t Python: \t{time_python:.5} seconds\n",
      f"\t C call: \t{time_C:.5} seconds\n",
      f"\t numpy: \t{time_numpy:.5} seconds\n")


Execution times for 100000 calls.
 	 Python: 	3.4561 seconds
 	 C call: 	0.17304 seconds
 	 numpy: 	0.25703 seconds



# What if the data types don't match?

In [7]:
# Create 1000 random floats between 0 and 1.  Only this time, create float32 words
arr = np.random.rand(1000).astype(np.float32)  # This is the "float" size on C

In [8]:
# Argument types
lib.find_max.argtypes = [ctypes.POINTER(ctypes.c_double), ctypes.c_int]

arr_ptr = arr.ctypes.data_as(ctypes.POINTER(ctypes.c_double))

In [9]:
# Call my C routine
max = lib.find_max(arr_ptr,len(arr))

# Compare it to the build in function
print(f"C function returns:\t{max}")
print(f"Built-in returns:\t{arr.max()}")

C function returns:	7.560971785098573e+305
Built-in returns:	0.9989924430847168


What if I change the definition of the pointer to match the array.

In [10]:
# Argument types
lib.find_max.argtypes = [ctypes.POINTER(ctypes.c_float), ctypes.c_int]

arr_ptr = arr.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

In [11]:
# Call my C routine
max = lib.find_max(arr_ptr,len(arr))

# Compare it to the build in function
print(f"C function returns:\t{max}")
print(f"Built-in returns:\t{arr.max()}")

C function returns:	7.560971785098573e+305
Built-in returns:	0.9989924430847168


It doesn't matter, because it will always just point to the beginning of the array